In [1]:
# Importing necessary modules
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as scs

In [2]:
##############################################################################
# Generating random draws of adjacency matrices from a given degree sequence
# Most of these codes are translations of Blitzstein's R codes.
##############################################################################

# Computes the corrected Durfee number of d
# Corrected Durfee number is needed to efficiently determine whether a
# degree sequence is graphical

def durfee(degrees):
    
    if N==1:
        return 1
        
    durfeeLB = 1
    durfeeUB = N
    m = int(np.floor((N+1)/2))
    
    while (durfeeLB<durfeeUB-1):
        
        if degrees[m-1]>=m-1:
            durfeeLB = m         
        else:
            durfeeUB = m-1
            
        m = int(np.floor((durfeeLB+durfeeUB)/2))
        
    if degrees[m]>=m:
        m = m+1
        
    return m
    

# Tests whether d is graphical using Erodos-Gallai Theorem

def isGraphical(degrees):   
    
    if max(degrees)>=N:
        return False     
        
    if (sum(degrees) %2)==1: # If sum is not even
        return False
        
    dsorted = sorted(degrees,reverse=True) # Sort in descending values
    
    if dsorted[N-1]<0:
        return False
        
    m = durfee(dsorted)
    minflips = [0]*N # minflips[0] records smallest indexed min. nonzero degree
                     # If smallest degree>0, minflips[0]=n.
                     # Let x1 be the next smallest degree.
                     # Then minflips[0:x1-1] will be the same (count=x1).
                     # The (x1+1)th item, minflips[x1] will be the smallest
                     # index with degree=x1 (call this index y1)
                     # Let x2 be the smallest degree bigger than x1
                     # minflips[x1:x2-1] will record y1 (count=x2-x1)
    
    if dsorted[N-1]>0:
        
        for j in range(0,dsorted[N-1]):
            minflips[j] = N # Python index begins from 0.
                            # The indices recorded in minflips stargs with 1
    for i in range(1,N):
        
        if dsorted[N-i-1]>dsorted[N-i]:
            
            for j in range(dsorted[N-i],dsorted[N-i-1]):
                minflips[j] = N-i
                
    partialsums = [dsorted[0]]
    
    for i in range(1,N):
        partialsums = partialsums+[partialsums[i-1]+dsorted[i]]
        
    for k in range(1,m+1):
        t = max(0,(minflips[k-1]-k)) # t is the number of degrees >=k
                                   # with indices >=k+1
        if partialsums[k-1]>(k*(k-1)+k*t+max(0,(partialsums[N-1]\
            -partialsums[k+t-1]))):
                
            return False
            
    return True


# Tests whether an edge x appears among the given edges

def isEdge(x,edges):
    
    edges = np.matrix(edges)
    m = edges.shape[0] # Number of rows in edges
    
    if m==0:
        return False
        
    for i in range(0,m):
        
        if (edges[i,0]==x[0] and edges[i,1]==x[1])\
            or (edges[i,0]==x[1] and edges[i,1]==x[0]): 
                
            return True
            
    return False
    

# Tests whether d with the ith and jth elements subtracted by 1 is graphical

def subtractGraphical(degrees,i,j):
    
    dtest = copy.copy(degrees)
    dtest[i] = dtest[i]-1
    dtest[j] = dtest[j]-1
    
    return isGraphical(dtest)
    

# Lists candidate vertices to connect vertex to, given a list of edges 
           
def listCandidates(vertex,degrees,edges):
    
    if (isGraphical(degrees)==False):
        return "Error: Non-graphical input in listCandidates"
        
    candidates = []
    
    for j in range(0,N):
        
        if (vertex!=j) and subtractGraphical(degrees,vertex,j)\
            and (isEdge([vertex,j],edges)==False):
            candidates = candidates+[j]
            
    return candidates
 
   
# Picks a new edge for vertex
   
def connectVertex(vertex,degrees,edges):
    
    cand = listCandidates(vertex,degrees,edges)
    numCand = len(cand)
    
    if numCand==0:
        return "Error: No candidates"
        
    candDegrees = [0]*numCand
    
    for i in range(0,numCand):
        candDegrees[i] = degrees[cand[i]]
        
    candDegrees = np.array(candDegrees).astype(float)
    p = (candDegrees/np.sum(candDegrees))
    j = np.random.choice(list(xrange(0,numCand)),1,False,p)
    w = cand[j]
    dnew = copy.copy(degrees)
    dnew[w] = dnew[w]-1
    dnew[vertex] = dnew[vertex]-1
    edgesnew = np.vstack((edges,[vertex,w]))
    
    return {'new_edge':[vertex,w],'prob':(candDegrees[j]/np.sum(candDegrees)),\
    'new_degrees':dnew,'candidates':cand,'candidateDegrees':candDegrees,\
    "new_edgelist":edgesnew}
  
  
# Picks edges for vertex until its residual degree is 0 
     
def connectVertexCompletely(vertex,degrees,edges):
    
    q = connectVertex(vertex,degrees,edges)
    dnew = q['new_degrees']
    seqProb = q['prob']
    edgesnew = q['new_edgelist']
    
    while dnew[vertex]>0:
        q = connectVertex(vertex,dnew,edgesnew)
        seqProb = seqProb*q['prob']
        dnew = q['new_degrees']
        edgesnew = q['new_edgelist']
        
    return {'sequentialProb':seqProb,'new_degrees':dnew,\
        'new_edgelist':edgesnew}


# Finds the index of the smallest nonzero entry in d    
    
def indexSmallestNonzero(degrees):
    
    m = 0
    
    for i in range(1,N):
        
        if (degrees[m] == 0) or ((degrees[i] != 0) and degrees[i]<degrees[m]):
            m = i
            
    return m


# Generates adjacency matrices from the result of generateGraphs()

def generateAdjs(result):
    
    mat = np.zeros((trials,N,N),dtype=int)
    
    for i in range(0,trials):
        graphs = result['graphs'][i]
        numEdges = len(graphs)
        
        for k in range(0,numEdges):
            mat[i,graphs[k,0],graphs[k,1]] = 1
            mat[i,graphs[k,1],graphs[k,0]] = 1
        
    return mat
        

# Generates a random graph using degree sequence d

def generateRandomGraph(degrees):
    
   if isGraphical(degrees)==False:
        return "Error: Non-graphical input in generateRandomGraph"
        
   degrees = copy.copy(degrees)
   seqProb = 1
   orders = 1
   edges = np.matrix([0,0])
   edges = np.delete(edges,0,axis=0)
   v = indexSmallestNonzero(degrees)
   vv = degrees[v]
   
   while vv>0:
       orders = orders*np.math.factorial(vv)
       q = connectVertexCompletely(v,degrees,edges)
       seqProb = seqProb*q['sequentialProb']
       degrees = q['new_degrees']
       edges = q['new_edgelist']
       v = indexSmallestNonzero(degrees)
       vv = degrees[v]
       
   seqProb = max(seqProb,minSeqProb)
   
   return {'N':N,'degrees':degrees,'sequentialProb':seqProb,\
   'numOrders':orders,'weight':(1./(seqProb*orders)),'edges':edges}


# Generates random graphs with degree sequence d and uses the
# results with importance sampling to estimate the number of graphs
# with degree sequence d
  
def generateGraphs(degrees):
    
    weights = [0]*trials
    graphs = {}
    
    for i in range(0,trials):  
        q = generateRandomGraph(degrees)
        weights[i] = q['weight']
        graphs[i] = q['edges']
        
    graphResults={'degrees':degrees,'weights':weights,'graphs':graphs,\
        'estimatedNumRealizations':(1./trials)*np.sum(weights),\
        'estimatedStardardError':(1./np.sqrt(trials))*np.std(weights)}
        
    return graphResults


# Computes triad census from an adjacency matrix

def triadCensus(adjM):
    
    adjM_new = np.matrix(adjM)
    tri = np.trace(adjM_new**3)/6
    two = (np.sum(adjM_new**2)-np.trace(adjM_new**2))/2-3*tri
    one = (N-2)*np.sum(adjM_new)/2-2*two-3*tri
    emp = N*(N-1)*(N-2)/6-tri-two-one
    
    return [emp,one,two,tri]


In [5]:
# Import urllib in order to download test data from Github repo
import urllib

# Download Nyakatoke test dataset from GitHub
# Edit to download location on your local   machine   
download =  '/Users/bgraham/Dropbox/Teaching/Short_Courses/St_Gallen/Data/Created/'                             
url = 'https://github.com/bryangraham/netrics/blob/master/Notebooks/Nyakatoke_Example.npz?raw=true'
urllib.urlretrieve(url, download + "Nyakatoke_Example.npz")

# Open dataset
NyakatokeTestDataset = np.load(download + "Nyakatoke_Example.npz")

# Extract adjacency matrix
Nyakatoke_adjM = NyakatokeTestDataset['D']

In [6]:
##############################################################################
# Nyakatoke Data
##############################################################################

## Loading Nyakatoke data

Nyakatoke_degrees = list(np.sum(Nyakatoke_adjM,axis=0,dtype=int))


## Generating adjacency matrices using Nyakatoke degree sequence

# Specifying global variables

N = len(Nyakatoke_degrees)
trials = 100
minSeqProb = 0.1**323 # This sets the minimum sequential probability for a
                      # random graph.
                      # 0.1**323 is the maximal precision Python allows.


# Generating random adjacency matrices

graph_result = generateGraphs(Nyakatoke_degrees) 
adj_result = generateAdjs(graph_result)


## Triad Census

#  Triad census of the actual Nyakatoke data

Nyakatoke_triads = triadCensus(Nyakatoke_adjM)


# Generating lists of triad census using the generated adjacency matrices

r_tri = np.zeros((trials),dtype=int)
r_two = r_tri.copy()
r_one = r_tri.copy()
r_emp = r_tri.copy()

for i in range(0,trials):
    q = triadCensus(adj_result[i,:,:])
    r_emp[i] = q[0]
    r_one[i] = q[1]
    r_two[i] = q[2]
    r_tri[i] = q[3]


## Hypothesis Testing with importance sampling

# Computing means using importance sampling

weights = np.array(graph_result['weights'])
mean_emp = (np.sum(weights*r_emp))/np.sum(weights)
mean_one = (np.sum(weights*r_one))/np.sum(weights)
mean_two = (np.sum(weights*r_two))/np.sum(weights)
mean_tri = (np.sum(weights*r_tri))/np.sum(weights)


# Computing fitted residual 

resid_emp = r_emp-mean_emp
resid_one = r_one-mean_one
resid_two = r_two-mean_two
resid_tri = r_tri-mean_tri


# Computing standard errors

denom = 1./np.sqrt(trials)*np.sum(weights)

se_emp = np.std(weights*resid_emp)/denom
se_one = np.std(weights*resid_one)/denom
se_two = np.std(weights*resid_two)/denom
se_tri = np.std(weights*resid_tri)/denom


# Computing Z-Scores and P-values

Z_emp = (mean_emp-Nyakatoke_triads[0])/se_emp
Z_one = (mean_one-Nyakatoke_triads[1])/se_one
Z_two = (mean_two-Nyakatoke_triads[2])/se_two
Z_tri = (mean_tri-Nyakatoke_triads[3])/se_tri
p_values = [1-scs.norm.cdf(Z_emp),scs.norm.cdf(Z_one),\
1-scs.norm.cdf(Z_two),scs.norm.cdf(Z_tri)]

 
# Creating summary table
 
r_triads_mean = [mean_emp,mean_one,mean_two,mean_tri]
r_triads_se = [se_emp,se_one,se_two,se_tri]
r_triads_summary = pd.DataFrame({'Mean':r_triads_mean,'Standard Error':\
r_triads_se},index=['Empties','One Edges','Two Stars','Threes'])
r_triads_summary.insert(0,'True Value',Nyakatoke_triads)
r_triads_summary.insert(3,'P-Value (one-sided)',p_values)
# Saving the summary as a .csv file
r_triads_summary.to_csv('Random_Nyakatoke_Triad_Census_Summary_%d_trials.csv'\
%trials)

print '\n'+'Summary Statistics of Triad Census from %d random draws' %trials
print r_triads_summary


## Histogram of triad census

# Putting data together in a single DataFrame

r_triad_census = pd.DataFrame({'Triangles':r_tri, 'Two Stars':r_two,\
'One Edges':r_one,'Empties (-220000)':r_emp-220000})
# Saving the random draws as a .csv file
r_triad_census.to_csv('Random_Triad_Census_%d_trials.csv' %trials )

shapes_list = r_triad_census.columns #Getting the list of shapes 


# Specifying figure size

fig = plt.figure()
fig.set_size_inches(8.5,11) # Letter size


# Plotting histograms

for i in range(0,4):
    plt.subplot(4,1,i+1) # First argument:  vertical dimension
                         # Second argument: horizontal dimension 
                         # Third argument:  location of the subplot
    plt.hist(r_triad_census[shapes_list[i]],bins=20) #Plot histogram (20 bins)
    plt.xlabel(shapes_list[i])
    plt.ylabel('Frequency')

plt.tight_layout() # Auto-adjusts subplot margins to avoid overlaps
plt.savefig('Triad_Census_%d_Trials.jpg' %trials) # Save to .jpg
plt.show()

/Users/bgraham/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:151: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


KeyboardInterrupt: 